In [ ]:
import sys
import os
!pwd
os.chdir('../../experiments/total-ab0-all')
sys.path.insert(0, '../../')
sys.path.insert(0, '../../mmdetection/')
import torch
print(torch)
import mmdet
print(mmdet)
import numpy as np
import cv2
import pickle
import mmcv
import torch
from scipy import ndimage
from torch.nn.functional import interpolate
from mmcv import Config
from curve.datasets import build_dataset, replace_ImageToTensor, build_dataloader
from curve.models import build_detector
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString, MultiLineString, GeometryCollection, Point
%matplotlib inline

In [ ]:
cfg = Config.fromfile('./config_demo.py')
cfg.gpus = 1

In [ ]:
model = build_detector(
        cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)

In [ ]:
def load_image(filename, scale=1):
    # image
    image = cv2.imread(filename, cv2.IMREAD_COLOR)
    image = cv2.resize(image, None, fx=scale, fy=scale)
    image = image[:, :, ::-1]
    return image

def load_mask(filename, path="Masks_gap", scale=1):
    # mask
    mask_filename = filename.replace('JPGImages', path).replace("jpg", "png")
    mask_gray = cv2.imread(mask_filename, cv2.IMREAD_GRAYSCALE)
    mask_gray = cv2.resize(mask_gray, None, fx=scale, fy=scale)
    _, gt_mask = cv2.threshold(mask_gray,127, 1,cv2.THRESH_BINARY)
    return gt_mask

def load_anns(filename, scale=1):
    ann_file = filename.replace('JPGImages', 'Annotations').replace("jpg", "txt")
    def _text_to_bboxes(text):
        x = text[0][5:-2].split()
        x = np.array(x, dtype=np.float32)
        y = text[1][6:-2].split()
        y = np.array(y, dtype=np.float32)
        box = np.hstack((x[:, np.newaxis], y[:, np.newaxis]))
        return box

    gt_bboxes=[]
    with open(ann_file, 'r') as f:
        content = f.read().splitlines()
        for text in content:
            pts = _text_to_bboxes(text.split(","))
            gt_bboxes.append((pts*scale).astype(np.int32))
    return gt_bboxes # list of [N, 2] numpy array


##################################
def draw_polygon(ax, bbox, color=[0, 1.0, 0], alpha=0.5, hatch=None, linewidth=2):
    import matplotlib
    polygon = matplotlib.patches.Polygon(
        bbox,
        fill=True,
        hatch=hatch,
        facecolor=list(color)+[alpha],
        edgecolor=list(color),
        linewidth=linewidth,
    )
    ax.add_patch(polygon)
##################################

colors = [[0.0, 0.44, 0.74], [0.85, 0.32, 0.09], [0.92, 0.69, 0.12], [0.49, 0.18, 0.55],
          [0.46, 0.67, 0.18], [0.3, 0.74, 0.93], [0.63, 0.07, 0.18], [0.6, 0.3, 0.3],
          [0.3, 0.6, 0.6], [1.0, 0.0, 0.0], [1.0, 0.5, 0.0], [0.74, 0.74, 0.0], [0.0, 1.0, 0.0],
          [0.0, 0.0, 1.0], [0.66, 0.0, 1.0], [0.33, 0.33, 0.0], [0.33, 0.66, 0.0], [0.33, 1.0, 0.0],
          [0.66, 0.33, 0.0], [0.66, 0.66, 0.0], [0.66, 1.0, 0.0], [1.0, 0.33, 0.0], [1.0, 0.66, 0.0],
          [1.0, 1.0, 0.0]]

In [ ]:
 # build the dataloader
samples_per_gpu = cfg.data.test.pop('samples_per_gpu', 1)
if samples_per_gpu > 1:
    # Replace 'ImageToTensor' to 'DefaultFormatBundle'
    cfg.data.test.pipeline = replace_ImageToTensor(cfg.data.test.pipeline)
print(samples_per_gpu)

In [ ]:
TRAIN=False
dataset = build_dataset(cfg.data.train if TRAIN else cfg.data.test)
data_loader = build_dataloader(
    dataset,
    samples_per_gpu=samples_per_gpu,
    workers_per_gpu=cfg.data.workers_per_gpu,
    dist=False,
    shuffle=False)

In [ ]:
from mmcv.runner.checkpoint import load_checkpoint
_ = load_checkpoint(model, './work_dirs/latest.pth',  strict=False, logger=None)

In [ ]:
model.CLASSES = dataset.CLASSES
_ = model.cuda()
_ = model.eval()
load_iter = data_loader.__iter__()
idx = 0

In [ ]:
from curve.tools.viz import image_from_tensor
exp_idx = 0
for i in range(idx//samples_per_gpu, exp_idx//samples_per_gpu):
    batch = load_iter.__next__()   
    idx+=samples_per_gpu

In [ ]:
while True:
    batch = load_iter.__next__()
    from mmcv.parallel.scatter_gather import scatter_kwargs
    inputs, kwargs = scatter_kwargs([], batch, target_gpus=[0], dim=0)
    inputs = inputs[0]
    kwargs = kwargs[0]
    key = ['img', 'img_metas']
    img, img_metas = [kwargs[i] for i in key]
    if TRAIN:
        for i in range(len(img_metas)):
            img_metas[i]['scale_factor'] = [img_metas[i]['scale_factor']]
    else:
        img = img[0]
        img_metas = img_metas[0]
    rescale = True
    print(img.shape)
    print("img idx:", idx, img_metas)
    idx+=len(img)
    with torch.no_grad():
        x = model.extract_feat(img)
        box_list, mask_list, gap_list, sdt_list, gapsdt_list, final_list = model.mask_head.simple_test_mask(x, img_metas, img.shape)


    ################################
    for i in range(len(box_list)):
        boxes = box_list[i]
        mask = mask_list[i]
        gap = gap_list[i]
        final = final_list[i]
        sdt = sdt_list[i]
        gapsdt = gapsdt_list[i]
        im = img[i][:, :mask.size(0), :mask.size(1)].cpu().numpy()
        im = np.transpose(im, (1, 2, 0))
        mean=np.array([[[123.675, 116.28, 103.53]]])
        mean = np.array(mean).reshape([1, 1, 3])
        var=np.array([[[58.395, 57.12, 57.375]]])
        var = np.array(var).reshape([1, 1, 3])
        im *= var
        im += mean
    #     if TRAIN:
    #         im = im[:, :, ::-1]
        im = im.astype(np.uint8)
        scale = img_metas[i]['scale_factor'][0]
        fig_h = int(32/7/im.shape[1]*im.shape[0])+1
        print(im.shape, fig_h)
        fig, ax = plt.subplots(1, 7, figsize=(32,fig_h), constrained_layout = True)
        [axi.set_axis_off() for axi in ax.ravel()]
        ax[0].imshow(mask)#im)
        # mask direct result
        ax[1].imshow(im) #final)
        binary_mask = (mask>0.5).numpy().astype("uint8")
        mask_contours, _ = cv2.findContours(binary_mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
        for idx, bbox in enumerate(mask_contours[::-1]):
            bbox = bbox.reshape((-1, 2))
            draw_polygon(ax[1], bbox, colors[idx%len(colors)])
        ax[2].imshow(gap) #final)
        ax[3].imshow(-sdt + 2*sdt*binary_mask)
        binary_gap = (gap>0.5).numpy().astype("uint8")
        ax[4].imshow(-gapsdt + 2*gapsdt*binary_gap)
        ax[5].imshow(final)
        # detections
        ax[6].imshow(im)
        for idx, box in enumerate(boxes):
            bbox = box[:-1].reshape((-1, 2))*scale
            draw_polygon(ax[6], bbox, colors[idx%len(colors)])
        filename = img_metas[i]['filename']
        filename = filename[filename.rfind('/')+1:]
        print(filename)
        fig.savefig('figures/%s'%filename, dpi=300)
        plt.close(fig)
    # img idx: 23 (23*4)

In [ ]:
for i in [0,]:
    boxes = box_list[i]
    mask = mask_list[i]
#     print(torch.max(mask), torch.min(mask))
    gap = gap_list[i]
#     print(torch.max(gap), torch.min(gap))
    final = final_list[i]
    sdt = sdt_list[i]
    im = img[i][:, :mask.size(0), :mask.size(1)].cpu().numpy()
    im = np.transpose(im, (1, 2, 0))
    mean=np.array([[[123.675, 116.28, 103.53]]])
    mean = np.array(mean).reshape([1, 1, 3])
    var=np.array([[[58.395, 57.12, 57.375]]])
    var = np.array(var).reshape([1, 1, 3])
    im *= var
    im += mean
    im = im.astype(np.uint8)
    scale = img_metas[i]['scale_factor'][0]
    fig, ax = plt.subplots(1, 2, figsize=(24,12))
    ax[0].imshow(im)
    ax[1].imshow(final)
#     mask[mask<0.1]=0
#     mask[mask>0.2]=1
#     ax[2].imshow(mask)
#     ax[3].imshow(gap)
#     ax[4].imshow(sdt)
#     colors = ['red', 'blue', 'green', 'orange', 'purple', 'lime', 'black']
    for j in range(len(boxes)):
        x = np.hstack([boxes[j][0:-1:2], boxes[j][0]])*scale
        y = np.hstack([boxes[j][1:-1:2], boxes[j][1]])*scale
        ax[0].plot(x, y, color=colors[j], linewidth=4)

In [ ]:
file_idx=0

In [ ]:
# vars
fig, ax = plt.subplots(2, 4, figsize=(16,8))
[axi.set_axis_off() for axi in ax.ravel()]
idx = 0
while idx<4:
    file_idx +=1
    filename="../../data/TotalText/JPGImages/%04d.jpg"%(file_idx)
    gt_gap = load_mask(filename, path="Masks_gap")
    if (gt_gap>0).sum() / (gt_gap>=0).sum()<0.01:
        continue
    print("%04d"%(file_idx))
    image = load_image(filename)
    gt_bboxes = load_anns(filename)
    # show
    ax[0,idx].imshow(image)
    ax[0,idx].imshow(image*0+255, alpha=0.5)
    for i, bbox in enumerate(gt_bboxes):
        draw_polygon(ax[0,idx], bbox, colors[i%len(colors)])
    #gap
    ax[1,idx].imshow(gt_gap)
    idx += 1

In [ ]:
# filename = "../../data/TotalText/JPGImages/0081.jpg"
filename = "../../data/TotalText/JPGImages/0290.jpg"
image = load_image(filename)
gt_bboxes = load_anns(filename)
gt_mask = load_mask(filename, path="Masks")
gt_gap = load_mask(filename, path="Masks_gap")

fig, ax = plt.subplots(2, 2, figsize=(20,20))
[axi.set_axis_off() for axi in ax.ravel()]
ax[0,0].imshow(image)
ax[0,1].imshow(image)
ax[0,1].imshow(image*0+255, alpha=0.3)
gt_polygon_list = []
for i, bbox in enumerate(gt_bboxes):    
    polygon = Polygon(bbox)
    radius = 4 #int(polylabel(polygon).distance(polygon.exterior) *0.4)
    polygon = polygon.buffer(radius, join_style=2)
    gt_polygon_list.append(polygon)
    bbox = np.array(polygon.exterior.xy).T.astype(np.int32)
    gt_bboxes_list.append(bbox)
    draw_polygon(ax[0,1], bbox, colors[i%len(colors)])
# gap contours
ax[1,0].imshow(image)
ax[1,0].imshow(image*0+255, alpha=0.3)
gap_contours, _ = cv2.findContours(gt_gap.astype("uint8"), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
for i, bbox in enumerate(gap_contours):
    draw_polygon(ax[1,0], bbox.reshape((-1, 2)), colors[i%len(colors)])
# final contours
ax[1,1].imshow(image)
ax[1,1].imshow(image*0+255, alpha=0.3)
gap_polys = [Polygon(box.squeeze()) for box in gap_contours]
for i, poly in enumerate(gt_polygon_list):
    for gap_poly in gap_polys:
        inter = poly.intersection(gap_poly)
        poly = poly.difference(inter.buffer(1))
    bbox = np.array(poly.exterior.xy).T.astype(np.int32)
    draw_polygon(ax[1,1], bbox.reshape((-1, 2)), colors[i%len(colors)])
# ax[1,1].imshow(image)
# ax[1,1].imshow(image*0+255, alpha=0.3)
# final_contours, _ = cv2.findContours((gt_mask*(1-gt_gap)).astype("uint8"), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
# for i, bbox in enumerate(final_contours[::-1]):
#     draw_polygon(ax[1,1], bbox.reshape((-1, 2)), colors[i%len(colors)])
# ax[3].imshow(gt_mask*(1-gt_gap))

In [ ]:
from curve.ops.polylabel import polylabel
# filename = "../../data/TotalText/JPGImages/0081.jpg"
filename = "../../data/TotalText/JPGImages/0435.jpg"
image = load_image(filename)
gt_bboxes = load_anns(filename)
gt_mask = load_mask(filename, path="Masks")
gt_gap = load_mask(filename, path="Masks_gap")

fig, ax = plt.subplots(2, 3, figsize=(20,20))
[axi.set_axis_off() for axi in ax.ravel()]
for axi in ax.ravel():
    axi.set_axis_off()
    axi.imshow(image)
    axi.imshow(image*0+255, alpha=0)
# show
ax[0,0].imshow(image)

colors = [[0.0, 0.44, 0.74], #orange red
          [0.85, 0.32, 0.09], # blue
          [0.46, 0.67, 0.18],
          [0.49, 0.18, 0.55],
          [0.92, 0.69, 0.12],
          [0.3, 0.74, 0.93], [0.63, 0.07, 0.18], [0.3, 0.3, 0.3],
          [0.6, 0.6, 0.6], [1.0, 0.0, 0.0], [1.0, 0.5, 0.0], [0.74, 0.74, 0.0], [0.0, 1.0, 0.0],
          [0.0, 0.0, 1.0], [0.66, 0.0, 1.0], [0.33, 0.33, 0.0], [0.33, 0.66, 0.0], [0.33, 1.0, 0.0],
          [0.66, 0.33, 0.0], [0.66, 0.66, 0.0], [0.66, 1.0, 0.0], [1.0, 0.33, 0.0], [1.0, 0.66, 0.0],
          [1.0, 1.0, 0.0]]
for i, bbox in enumerate(gt_bboxes[::-1]):
    draw_polygon(ax[0,1], bbox, colors[i%len(colors)], alpha=0.7)
    draw_polygon(ax[1,2], bbox, colors[i%len(colors)], alpha=0.3, hatch='.', linewidth=2)
    polygon = Polygon(bbox)
    radius = int(polylabel(polygon).distance(polygon.exterior) *0.4)    
    bbox = np.array(polygon.buffer(radius).exterior.xy).T.astype(np.int32)
    draw_polygon(ax[0,2], bbox, colors[i%len(colors)], alpha=0.9)
# gap contours
gap_contours, _ = cv2.findContours(gt_gap.astype("uint8"), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
for i, bbox in enumerate(gap_contours):
    draw_polygon(ax[1,0], bbox.reshape((-1, 2)), colors[i%len(colors)+3], alpha=0.7)
    draw_polygon(ax[0,2], bbox.reshape((-1, 2)), colors[i%len(colors)+3], alpha=0.9)
    draw_polygon(ax[1,2], bbox.reshape((-1, 2)), colors[i%len(colors)+3], alpha=0.3, hatch='-', linewidth=2)
# final contours
final_contours, _ = cv2.findContours((gt_mask*(1-gt_gap)).astype("uint8"), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
for i, bbox in enumerate(final_contours[::-1]):
    draw_polygon(ax[1,1], bbox.reshape((-1, 2)), colors[i%len(colors)], alpha=0.7)
# ax[3].imshow(gt_mask*(1-gt_gap))

In [ ]:
# SDT

In [ ]:
# sdt
def get_sdt(gt_mask):
    from scipy import ndimage
    gt_mask = gt_mask.astype(np.uint8)
    kernel = np.ones((3,3), np.uint8)
    contour_map = cv2.dilate(gt_mask, kernel, iterations=1) - gt_mask
    dt = ndimage.distance_transform_edt(contour_map == 0)
    sdt = dt.copy() #phi_0
    thres = 40.0
    sdt[sdt > thres] = thres  # truncate
    sdt[sdt < -thres] = - thres  # truncate
    sdt[gt_mask <= 0] *= -1
    return sdt/thres


# filename = "../../data/TotalText/JPGImages/0081.jpg"
filename = "../../data/TotalText/JPGImages/0656.jpg"
scale = 2
image = load_image(filename, scale=scale)
print(image.shape)
gt_bboxes = load_anns(filename, scale=scale)
gt_mask = load_mask(filename, path="Masks", scale=scale)
gt_gap = load_mask(filename, path="Masks_gap", scale=scale)

fig, ax = plt.subplots(2, 2, figsize=(20,20))
[axi.set_axis_off() for axi in ax.ravel()]
for axi in ax.ravel():
    axi.set_axis_off()
    axi.imshow(image)
    axi.imshow(image*0+255, alpha=0.3)
# show
for i, bbox in enumerate(gt_bboxes[::-1]):
    draw_polygon(ax[0,0], bbox, colors[i%len(colors)], alpha=0.7)
# gap contours
gap_contours, _ = cv2.findContours(gt_gap.astype("uint8"), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
for i, bbox in enumerate(gap_contours):
    draw_polygon(ax[1,0], bbox.reshape((-1, 2)), colors[i%len(colors)], alpha=0.9)
# ax[3].imshow(gt_mask*(1-gt_gap))
# mask sdt
mask_sdt = get_sdt(gt_mask)
ax[0, 1].imshow(mask_sdt)
# gap sdt
gap_sdt = get_sdt(gt_gap)
ax[1, 1].imshow(gap_sdt)

In [ ]:
a = np.ones(gt_gap.shape)
a.shape

In [ ]:
filename="../../data/TotalText/JPGImages/%04d.jpg"%(file_idx*4+2)
gt_gap = load_mask(filename, path="Masks_gap")
kernel = np.ones((3,3),np.uint8)
contour_map = cv2.dilate(gt_gap.astype(np.uint8), kernel, iterations=1)-gt_gap
# contours, _ = cv2.findContours(gt_gap.astype("uint8"), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
# contour_map = cv2.drawContours(gt_gap.astype("uint8")*0, contours, -1, 255, 1)
print(contour_map.max(), np.sum(contour_map>0))
dt = ndimage.distance_transform_edt(contour_map==0)
sdt = dt.copy()
thres = 40.0
sdt[sdt > thres] = thres
sdt[gt_gap > 0] *= -1
print(filename, sdt.min(), sdt.max())
fig, ax = plt.subplots(1, 3, figsize=(24,8))
ax[0].imshow(gt_gap)
ax[1].imshow(contour_map)
ax[2].imshow(sdt)

In [ ]:
H, W = 10, 10
x = torch.linspace(-1.0, 1.0, W)
y = torch.linspace(-1.0, 1.0, H)
xx = x.repeat(len(y))
yy = y.view(-1, 1).repeat(1, len(x)).view(-1)
positions = torch.stack([xx, yy], dim=-1)
x = torch.rand(3, 3)
print(x)
x.view(-1)

In [ ]:
x = torch.rand(200, 2)
x = torch.nn.functional.normalize(x, dim=-1, p=2)
cl, c = KMeans(x, K=2, Niter=3, verbose=True)
torch.unique(cl)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
cdict = ['b','g','r','c','m','y','k','w']
fig, ax = plt.subplots()
for g in torch.unique(cl):
    ix = torch.where(cl == g)[0]
    ax.scatter(x[ix, 0], x[ix, 1], c = cdict[int(g)])

In [ ]:
x = torch.rand(3, 2)
x

In [ ]:
torch.nn.functional.normalize(x*9, dim=-1, p=2)

In [ ]:
## Step #1
from matplotlib import cm
import numpy as np
N = 100 # You define this here
colours = cm.get_cmap('Accent', N)  # Change the string from 'viridis' to whatever you want from the above link
cmap = colours(np.linspace(0, 1, N))  # Obtain RGB colour map
cmap[:,-1] = 1  # Set alphas for the labels to be 0.3
labels = cluster.int()
print(cmap.shape, labels.shape)
## Step #2
output = cmap[labels.flatten()]
## Step #3
R, C = cluster.shape[:2]
output = output.reshape((R, C, -1))
print(output.shape)
## Overlay
fig, ax = plt.subplots(1, 2, figsize=(12,12))
ax[0].imshow(im)
ax[1].imshow(output)